# **1. Business Understanding**

blablabla.....................

**Tujuan**
* Memprediksi jenis pelanggan yang berpotensi melakukan churn dengan mengidentifikasi fitur-fitur untuk meminimalkan tingkat churn pelanggan dan mendapatkan keputusan bisnis yang tepat.

**Sasaran**
*   Membuat model untuk memprediksi pengguna yang berpotensi melakukan churn dan memahami fitur apa saja yang menyebabkan pengguna melakukan churn serta fitur yang memacu potensi untuk meminimalisir tingkat churn.

*   blabla




note -> bole diganti klo emng kurang ya

# **Import Library**

In [1]:
import pandas as pd
import numpy as np
import datetime
import ast
from ast import literal_eval
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report, roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

# **2. Data Preparation**

## Load Dataset

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!unzip drive/MyDrive/dataset/SAMPLE/data.zip
#!unzip drive/MyDrive/dataset/dataset.zip

In [ ]:
# click = pd.read_csv('/content/drive/MyDrive/dataset/SAMPLE/click_stream_1_percent_sample.csv')
# customer = pd.read_csv('/content/drive/MyDrive/dataset/SAMPLE/customer_1_percent_sample.csv')
# product = pd.read_csv('/content/drive/MyDrive/dataset/SAMPLE/product_1_percent_sample.csv', usecols=['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName'])
# transactions = pd.read_csv('/content/drive/MyDrive/dataset/SAMPLE/transaction_1_percent_sample.csv')

#click = pd.read_csv('/content/drive/MyDrive/dataset/click_stream.csv')
#customer = pd.read_csv('/content/drive/MyDrive/dataset/customer.csv')
#product = pd.read_csv('/content/drive/MyDrive/dataset/product.csv', usecols=['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName'])
#transactions = pd.read_csv('/content/drive/MyDrive/dataset/transactions.csv')

In [2]:
click = pd.read_csv('/home/faris/dataset/click_stream.csv')
customer = pd.read_csv('/home/faris/dataset/customer.csv')
product = pd.read_csv('/home/faris/dataset/product.csv', usecols=['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName'])
transactions = pd.read_csv('/home/faris/dataset/transactions.csv')

In [3]:
pd.set_option('display.max_columns', None)

## EDA ( Exploratory Data  Analysis)

Data Wrangling

### Transaction Dataset

In [4]:
# mengubah tipe data ke kategori dan date
transactions[['payment_method', 'payment_status', 'promo_code']] = transactions[['payment_method', 'payment_status', 'promo_code']].astype('category')
transactions['shipment_date_limit'] = pd.to_datetime(transactions['shipment_date_limit'])
transactions['created_at'] = pd.to_datetime(transactions['created_at'])

In [5]:
# parsing metadata dari kolom product_metadata
transactions['product_metadata'] = transactions['product_metadata'].fillna('{}')
transactions['product_metadata'] = transactions['product_metadata'].apply(literal_eval)
transactions = ( transactions.explode('product_metadata').reset_index(drop=True))
transactions = pd.concat([transactions.drop(['product_metadata'], axis=1), pd.json_normalize(transactions['product_metadata'])], axis=1)

transactions.head()

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price
0,2018-07-29 15:22:01.458193+00:00,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03 05:07:24.812676+00:00,-8.227893,111.969107,199832,54728,1,191247
1,2018-07-30 12:40:22.365620+00:00,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03 01:29:03.415705+00:00,3.013470,107.802514,155526,16193,1,145526
2,2018-09-15 11:51:17.365620+00:00,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18 08:41:49.422380+00:00,-2.579428,115.743885,550696,53686,4,135174
3,2018-11-01 11:23:48.365620+00:00,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05 17:42:27.954235+00:00,-3.602334,120.363824,271012,20228,1,271012
4,2018-12-18 11:20:30.365620+00:00,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23 17:24:07.361785+00:00,-3.602334,120.363824,198753,55220,1,198753


In [6]:
transactions.shape

(1254585, 16)

In [7]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254585 entries, 0 to 1254584
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype              
---  ------                  --------------    -----              
 0   created_at              1254585 non-null  datetime64[ns, UTC]
 1   customer_id             1254585 non-null  int64              
 2   booking_id              1254585 non-null  object             
 3   session_id              1254585 non-null  object             
 4   payment_method          1254585 non-null  category           
 5   payment_status          1254585 non-null  category           
 6   promo_amount            1254585 non-null  int64              
 7   promo_code              481139 non-null   category           
 8   shipment_fee            1254585 non-null  int64              
 9   shipment_date_limit     1254585 non-null  datetime64[ns, UTC]
 10  shipment_location_lat   1254585 non-null  float64            
 11  shipment_lo

In [8]:
transactions.describe()

,customer_id,promo_amount,shipment_fee,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price
count,1.254585e+06,1.254585e+06,1.254585e+06,1.254585e+06,1.254585e+06,1.254585e+06,1.254585e+06,1.254585e+06,1.254585e+06
mean,4.980987e+04,2.046875e+03,9.190860e+03,-5.111930e+00,1.109800e+02,1.181574e+06,2.970114e+04,1.475128e+00,2.499744e+05
std,2.900057e+04,3.068618e+03,9.371999e+03,3.080204e+00,6.400801e+00,1.728940e+06,1.705351e+04,1.590220e+00,1.118733e+05
min,3.000000e+00,0.000000e+00,0.000000e+00,-1.099552e+01,9.503073e+01,1.089800e+04,1.163000e+03,1.000000e+00,5.600000e+03
25%,2.447800e+04,0.000000e+00,0.000000e+00,-7.375628e+00,1.068633e+02,2.466450e+05,1.477600e+04,1.000000e+00,1.683730e+05
50%,4.957700e+04,0.000000e+00,1.000000e+04,-6.242058e+00,1.101724e+02,4.558770e+05,2.861400e+04,1.000000e+00,2.335400e+05
75%,7.494800e+04,3.998000e+03,1.000000e+04,-3.090132e+00,1.132200e+02,1.360496e+06,4.469500e+04,1.000000e+00,3.136470e+05
max,9.999900e+04,2.451900e+04,5.000000e+04,5.874791e+00,1.410061e+02,2.350449e+07,6.000000e+04,4.800000e+01,1.200357e+06


In [9]:
transactions.isnull().sum()

created_at                     0
customer_id                    0
booking_id                     0
session_id                     0
payment_method                 0
payment_status                 0
promo_amount                   0
promo_code                773446
shipment_fee                   0
shipment_date_limit            0
shipment_location_lat          0
shipment_location_long         0
total_amount                   0
product_id                     0
quantity                       0
item_price                     0
dtype: int64

In [10]:
transactions.duplicated().sum()

0

### Click Stream Dataset

In [12]:
# mengubah tipe data ke kategori dan date
click[['event_name', 'traffic_source']] = click[['event_name', 'traffic_source']].astype('category')
click['event_time'] = pd.to_datetime(click['event_time'])

In [13]:
# parsing metadata dari kolom event_metadata
def parse_event_metadata(row):
  if pd.isnull(row) or row == '':
    return {}
  return ast.literal_eval(row)

click['event_metadata'] = click['event_metadata'].apply(parse_event_metadata)

metadata_keys = set()

def extract_keys(row):
  for key in row.keys():
    metadata_keys.add(key)

click['event_metadata'].apply(extract_keys)
metadata_keys

{'item_price',
 'payment_status',
 'product_id',
 'promo_amount',
 'promo_code',
 'quantity',
 'search_keywords'}

In [ ]:
click['product_id'] = click['event_metadata'].apply(lambda x: x.get('product_id'))
click['quantity'] = click['event_metadata'].apply(lambda x: x.get('quantity'))
click['item_price'] = click['event_metadata'].apply(lambda x: x.get('item_price'))
click['payment_status'] = click['event_metadata'].apply(lambda x: x.get('payment_status'))
click['product_amount'] = click['event_metadata'].apply(lambda x: x.get('product_amount'))
click['promo_code'] = click['event_metadata'].apply(lambda x: x.get('promo_code'))
click['search_keywords'] = click['event_metadata'].apply(lambda x: x.get('search_keywords'))
# Menghapus kolom event_metadata asli
click.drop(columns=['event_metadata'], inplace=True)
click.head()

In [ ]:
click_selected = ['event_name', 'traffic_source', 'payment_status', 'promo_code', 'search_keywords']

for col_name in click_selected:
    print(col_name)
    print(click[col_name].unique())
    print('\n')

In [ ]:
# mengubah tipe data kategori
click[['payment_status', 'search_keywords', 'promo_code']] = click[['payment_status', 'search_keywords', 'promo_code']].astype('category')

In [ ]:
click.info()

In [ ]:
click.shape

In [ ]:
click.describe()

In [ ]:
click.isnull().sum()

In [ ]:
click.duplicated().sum()

### Customer Dataset

In [ ]:
customer.head()

In [ ]:
customer.info()

In [ ]:
# mengubah tipe data kategori dan date
customer[['gender','device_type', 'device_version', 'home_location', 'home_country']] = customer[['gender','device_type', 'device_version', 'home_location', 'home_country']].astype('category')
customer['birthdate'] = pd.to_datetime(customer['birthdate'])
customer['first_join_date'] = pd.to_datetime(customer['first_join_date'])

In [ ]:
customer.info()

In [ ]:
customer.describe()

In [ ]:
customer.isnull().sum()

In [ ]:
customer.duplicated().sum()

### Product Dataset

In [ ]:
product.head()

In [ ]:
product.rename(columns={'id': 'product_id'}, inplace=True)

In [ ]:
# mengubah tipe data kategori dan date
product[['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage', 'productDisplayName']] = product[['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage', 'productDisplayName']].astype('category')
product['year'] = pd.to_datetime(product['year'])


In [ ]:
product.info()

In [ ]:
product.describe()

In [ ]:
product.isnull().sum()

In [ ]:
product.duplicated().sum()

In [ ]:
transactions['payment_method'].unique()

In [ ]:
transactions['payment_status'].unique()

### Data  Visualization

In [ ]:
# #Melihat Persebaran Customer
# customer['home_location'].value_counts()[0:].plot(kind='bar')

In [ ]:
# #Melihat Persebaran Umur Customer
# customer['umur'] = 2022-customer['birthdate'].dt.year
# sns.distplot(customer['umur'])
# plt.xlabel("umur customer")
# plt.show()

In [ ]:
# #Melihat Peresentase Persebaran Gender Customer
# x = ["Female", "Male"]
# a = customer[customer["gender"] == "F"]["gender"].count()
# b = customer[customer["gender"] == "M"]["gender"].count()
# y = np.array([a, b])
# plt.figure(figsize=(5,4))
# bar_colors = ['tab:red', 'tab:blue']
# plt.pie(y, labels = x, colors = bar_colors, autopct='%1.1f%%')
# plt.title("Persentase Gender Customer")
# plt.show()

Melihat distribusi device yang digunakan pelanggan

In [ ]:
# # Sumber trafik
# traffic_source_counts = click['traffic_source'].value_counts()

# # Membuat visualisasi dengan grafik batang
# plt.figure(figsize=(5, 3))
# plt.bar(traffic_source_counts.index, traffic_source_counts.values)
# plt.xlabel('Traffic Source')
# plt.ylabel('Number of Events')
# plt.title('Traffic Source Distribution')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

In [ ]:
# #Melihat Presentase Tipe Device Customer
# plt.figure(figsize=(6,4))
# plt.pie(customer['device_type'].value_counts(),
#         autopct="%1.1f%%",
#         labels=customer['device_type'].value_counts().index)
# plt.title("Persentase Tipe Device Customer")
# plt.show()

In [ ]:
# # Melihat Jumlah Pelanggan Yang Bergabung Berdasarkan Tahun dan Bulan
# date = customer.copy()
# date['first_join_date'] = date['first_join_date'].astype('datetime64')

# # partisi berdasarkan year dan month
# date['year_join'] = date['first_join_date'].dt.year
# date['month_join'] = date['first_join_date'].dt.month

# month = date.groupby(['year_join', 'month_join']).agg(
#     customers_join = pd.NamedAgg(column = 'month_join', aggfunc = sum))
# month.reset_index(inplace=True)

# sns.set(rc={"figure.figsize":(9, 4)})
# sns.lineplot(data=month, x="month_join", y="customers_join", hue="year_join", palette="deep")

In [ ]:
# #Melihat Presentase Kategori Terlaris
# customer_product=transactions.merge(product[['product_id','masterCategory','subCategory','articleType','season']],on='product_id',how='left')
# customer_product=customer_product.merge(customer[['customer_id','gender']],on='customer_id',how='left')

# plt.figure(figsize=(6,4))
# plt.pie(customer_product['masterCategory'].value_counts()[:5],
#         autopct="%1.1f%%",
#         labels=customer_product['masterCategory'].value_counts()[:5].index)
# plt.title("Persentase Kategori Terlaris")
# plt.show()

In [ ]:
# # Jenis pakaian yang paling banyak diminati oleh customer

# topwear = customer_product[customer_product['subCategory'] == 'Topwear']
# topwear.rename(columns = {"articleType":"Clothes"},inplace=True)
# plt.figure(figsize=(10,5))
# sns.countplot(x = 'Clothes',
#               data = topwear,
#               order = topwear['Clothes'].value_counts()[:10].index)
# plt.title('Penjualan Pakaian Terlaris')

In [ ]:
# #Melihat Presentasi Payment Method
# plt.figure(figsize=(6,4))
# plt.pie(transactions["payment_method"].value_counts(),
#         autopct="%1.1f%%",
#         labels=transactions['payment_method'].value_counts().index)
# plt.title("Persentase Payment Method")
# plt.show()

In [ ]:
# plt.figure(figsize=(6,4))
# plt.pie(click["traffic_source"].value_counts(),
#         autopct="%1.1f%%",
#         labels=click['traffic_source'].value_counts().index)
# plt.title("Persentase Traffic Source")
# plt.show()

Menganalisa presentase pelanggan yang sukses melakukan pembelian

In [ ]:
# # Memfilter Transactions yang Memiliki Payment Status = Success
# success = transactions.copy()
# success = transactions[transactions['payment_status'] == 'Success']

# total = customer['customer_id'].nunique() #Jumlah pelanggan terdaftar
# trans_sukses = success['customer_id'].nunique() #Jumlah pelanggan yang memiliki setidaknya satu transaksi sukses
# no_trans = total-trans_sukses # Jumlah pelanggan yang belum melakukan transaksi.

# print('Jumlah customers terdaftar:', total)
# print('Jumlah customers yang pernah bertransaksi:', transactions['customer_id'].nunique())
# print('Jumlah customers yang sukses bertransaksi:', trans_sukses)

# # Memastikan bahwa nilai-nilai di dalam array y bersifat non-negatif
# x = ["Belum bertransaksi", "Sukses bertransaksi"]
# y = np.array([no_trans, trans_sukses])  # Memastikan bahwa c diubah menjadi positif

# bar_colors = ['tab:red', 'tab:blue']

# # Membuat diagram lingkaran dengan sektor "Sukses bertransaksi" muncul terlebih dahulu
# plt.figure(figsize=(6, 4))
# plt.pie(y, labels=x, colors=bar_colors, autopct='%1.1f%%', startangle=90, counterclock=False)
# plt.title('Jumlah Customers')
# plt.show()


In [ ]:
# # transactions by promo usage
# date_trans = success['created_at'].nunique()
# use_code = success['promo_code'].value_counts().sum()
# without_code = date_trans-use_code

# print('Jumlah Transaksi:', date_trans)
# print('1. Dengan promo:', use_code)
# print('2. Tanpa promo:', without_code)

# x = ["Dengan promo", "Tanpa promo"]
# y = np.array([use_code, without_code])
# bar_colors = ['tab:blue', 'tab:red']

# plt.figure(figsize=(6,5))
# plt.pie(y, labels = x, colors = bar_colors, autopct='%1.1f%%')
# plt.title('Jumlah Transaksi')
# plt.show()

In [ ]:
# # transactions by promo_code usage
# success['promo_code'].value_counts().sort_values().plot(kind='barh', color='skyblue', figsize=(8, 4))
# plt.xlabel('Jumlah Transaksi')
# plt.ylabel('Kode Promo')
# plt.title('Jumlah Transaksi berdasarkan Penggunaan Kode Promo')
# plt.show()

In [ ]:
# # transactions by created_at
# date = success.copy()
# date['created_at'] = date['created_at'].astype('datetime64')
# date['year_transaction'] = date['created_at'].dt.year # partisi berdasarkan year
# date['month_transaction'] = date['created_at'].dt.month # partisi bedasarkan month

# month = date.groupby(['year_transaction', 'month_transaction']).agg(
#     customers_transaction = pd.NamedAgg(column = 'month_transaction', aggfunc = sum))  # membuat kolom customers_transaction
# month.reset_index(inplace=True)

# sns.set(rc={"figure.figsize":(9, 4)})
# sns.lineplot(data=month, x="month_transaction", y="customers_transaction", hue="year_transaction", palette="deep")

In [ ]:
# Melihat Booking Yang Pernah Dilakukan Customer
# booking = success.groupby(['customer_id']).agg(
#     customer_booking = pd.NamedAgg(column = 'booking_id', aggfunc = len))
# booking.reset_index(inplace=True)

# print(booking['customer_booking'].describe().round(0))
# sns.displot(booking, x="customer_booking", binwidth=18)

In [ ]:
# # shipment fee
# print(success['shipment_fee'].describe().round(0))
# sns.displot(success, x="shipment_fee")

In [ ]:
# #Presentase Penjualan(sukses) Berdasarkan Musim
# success = transactions.copy()
# success = transactions[transactions['payment_status'] == 'Success']

# product_transactions = success.merge(product, how='left', on='product_id') # merge column transactions dan products
# season = product_transactions.groupby('season')['quantity'].sum()
# plt.figure(figsize=(6,4))
# plt.pie(season.sort_values(ascending=False),
#         autopct="%1.1f%%",
#         labels=season.sort_values(ascending=False).index)
# plt.title("Persentase Periode dengan Penjualan Terlaris")
# plt.show()

In [ ]:
# #Presentase Penjualan(sukses) Berdasarkan Gender Produk
# gender = product_transactions.groupby('gender')['quantity'].sum()
# plt.figure(figsize=(6,4))
# plt.pie(gender.sort_values(ascending=False),
#         autopct="%1.1f%%",
#         labels=gender.sort_values(ascending=False).index)
# plt.title("Persentase Jenis Gender Produk Terlaris")
# plt.show()

In [ ]:
# # Melihat Presentase Produk Terlaris Dari Fungsinya
# usage = product_transactions.groupby('usage')['quantity'].sum()
# plt.figure(figsize=(6,4))
# plt.pie(usage.sort_values(ascending=False)[:5],
#         autopct="%1.1f%%",
#         labels=usage.sort_values(ascending=False)[:5].index)
# plt.title("Persentase Fungsi dari Produk Terlaris")
# plt.show()

In [ ]:
# # Melihat Presentase Produk Terlaris Dari SubCategory
# subCategory = product_transactions.groupby('subCategory')['quantity'].sum()
# plt.figure(figsize=(6,4))
# subCategory.sort_values(ascending=False)[:10].sort_values().plot(kind='barh')

In [ ]:
# # Melihat Presentase Produk Terlaris Dari BaseColour
# baseColour = product_transactions.groupby('baseColour')['quantity'].sum()
# plt.figure(figsize=(6,4))
# baseColour.sort_values(ascending=False)[:10].sort_values().plot(kind='barh')

In [ ]:
# # Melihat Presentase Produk Terlaris Dari ArticleType
# articleType = product_transactions.groupby('articleType')['quantity'].sum()
# plt.figure(figsize=(6,4))
# articleType.sort_values(ascending=False)[:10].sort_values().plot(kind='barh')

In [ ]:
# # Konversi kolom 'created_at' ke tipe datetime
# transactions['created_at'] = pd.to_datetime(transactions['created_at'])

# # Tahun-tahun yang ingin dianalisis
# years_to_analyze = [2017, 2018, 2019, 2020, 2021, 2022]

# for year in years_to_analyze:
#     # Filter data berdasarkan tahun
#     transactions_year = transactions[transactions['created_at'].dt.year == year]

#     # Hitung jumlah pengunjung yang berhasil melakukan pembayaran per bulan
#     successful_payments_per_month = transactions_year[transactions_year['payment_status'] == 'Success']\
#         .groupby(transactions_year['created_at'].dt.month)['customer_id'].nunique()

#     # Hitung jumlah pengunjung secara keseluruhan per bulan
#     total_visitors_per_month = transactions_year.groupby(transactions_year['created_at'].dt.month)['customer_id'].nunique()

#     # Hitung conversion rate per bulan
#     conversion_rate_per_month = (successful_payments_per_month / total_visitors_per_month).fillna(0)

#     # Plot data conversion rate
#     plt.figure()
#     conversion_rate_per_month.plot(kind='line', xlabel='Bulan', ylabel='Conversion Rate')
#     plt.title(f'Conversion Rate per Bulan pada Tahun {year}')

# plt.show()

In [ ]:
# # Filter successful transactions
# successful_transactions = transactions[transactions['payment_status'].isin(['success', 'completed'])]

# # Calculate the unique number of customers
# unique_customers = transactions['customer_id'].nunique()

# # Calculate the number of successful transactions
# num_successful_transactions = successful_transactions.shape[0]

# # Calculate the conversion rate
# conversion_rate = (num_successful_transactions / unique_customers) * 100

# print(f'Conversion Rate: {conversion_rate:.2f}%')

# # Group transactions by year
# transactions['created_at'] = pd.to_datetime(transactions['created_at'])
# transactions['year'] = transactions['created_at'].dt.year
# conversion_by_year = transactions.groupby('year')['payment_status'].count()

# # Plot the conversion rate by year
# plt.figure(figsize=(10, 6))
# plt.bar(conversion_by_year.index, conversion_by_year.values)
# plt.xlabel('Year')
# plt.ylabel('Number of Successful Transactions')
# plt.title('Conversion Rate by Year')
# plt.show()

Melihat presentase barang yang dimasukan ke keranjang oleh pelanggan, tapi tidak melanjutkan ke proses pembayaran

In [ ]:
# # Menghitung jumlah transaksi tanpa pembayaran
# abandoned_carts = transactions[transactions["payment_status"] == "unpaid"]
# total_transactions = len(transactions)

# # Menghitung Cart Abandonment Rate
# abandonment_rate = (len(abandoned_carts) / total_transactions) * 100

# # Membuat visualisasi
# labels = ['Completed Transactions', 'Abandoned Carts']
# sizes = [total_transactions - len(abandoned_carts), len(abandoned_carts)]
# colors = ['lightblue']
# explode = (0, 0.1)  # Pecahkan slice 'Abandoned Carts'

# plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
# plt.axis('equal')  # Mengatur aspek lingkaran menjadi lingkaran
# plt.title('Cart Abandonment Rate')
# plt.show()

# print(f"Cart Abandonment Rate: {abandonment_rate:.2f}%")

In [ ]:
# transactions['payment_status'].unique()

In [ ]:
# # Menghitung jumlah setiap nilai dalam 'payment_status'
# payment_status_counts = click['payment_status'].value_counts()

# # Menampilkan hasil
# print("Jumlah dan Jenis Status Pembayaran:")
# print(payment_status_counts)

# # Membuat plot
# plt.figure(figsize=(6, 4))
# payment_status_counts.plot(kind='bar', color='pink')
# plt.xlabel('Payment Status')
# plt.ylabel('Jumlah Transaksi')
# plt.title('Diagram Click Payment Status')
# plt.xticks(rotation=0)
# plt.show()

In [ ]:
# # Menghitung jumlah payment method dengan status 'success'
# success_count = transactions[transactions['payment_status'] == 'Success']['payment_method'].value_counts()

# # Menghitung jumlah payment method dengan status 'failed'
# failed_count = transactions[transactions['payment_status'] == 'Failed']['payment_method'].value_counts()

# print("Jumlah payment method dengan status 'success':")
# print(success_count)

# print("\nJumlah payment method dengan status 'failed':")
# print(failed_count)

In [ ]:
# # Menghitung jumlah dan kata kunci (search keywords)
# search_keywords_counts = click['search_keywords'].value_counts()

# # Menampilkan hasil
# print("Jumlah dan Kata Kunci:")
# print(search_keywords_counts)

In [ ]:
# # Mengetahui jumlah transactions dari seluruh customers

# # Menghitung jumlah pembelian per pelanggan di dataset transactions
# total_purchase_counts = transactions.groupby('customer_id').size().reset_index(name='total_purchase_count')

# # Gabungkan hasil dengan data customer menggunakan 'outer'
# result_customer = pd.merge(customer, total_purchase_counts, on='customer_id', how='outer')

# # Mengganti nilai NaN pada kolom 'total_purchase_count' dengan 0
# result_customer['total_purchase_count'].fillna(0, inplace=True)

# # Mengubah tipe data
# result_customer['total_purchase_count'] = result_customer['total_purchase_count'].astype(int)

# # Mengurutkan hasil berdasarkan 'total_purchase_count'
# result_customer = result_customer.sort_values(by='total_purchase_count',ascending=[0])

# # Menyusun ulang indeks jika diperlukan
# result_customer = result_customer.reset_index(drop=True)

# # Menampilkan hasil
# result_customer[['customer_id','first_name','last_name','gender','total_purchase_count']]

In [ ]:
# # Menghitung total pembelian per pelanggan di dataset transactions
# total_purchase_amount = transactions.groupby('customer_id')['total_amount'].sum().reset_index(name='total_purchase_amount')

# # Gabungkan hasil dengan data customer menggunakan 'outer'
# result_customer = pd.merge(customer, total_purchase_amount, on='customer_id', how='outer')

# # Mengganti nilai NaN pada kolom 'total_purchase_amount' dengan 0
# result_customer['total_purchase_amount'].fillna(0, inplace=True)

# # Mengubah tipe data
# result_customer['total_purchase_amount'] = result_customer['total_purchase_amount'].astype(int)

# # Mengurutkan hasil berdasarkan 'customer_id'
# result_customer = result_customer.sort_values(by='total_purchase_amount',ascending=[0])

# # Menyusun ulang indeks jika diperlukan
# result_customer = result_customer.reset_index(drop=True)

# # Menampilkan hasil
# result_customer[['customer_id','first_name','last_name','gender','total_purchase_amount']]

In [ ]:
# # Menampilkan productDisplay Name dan baseColour ke dataset transactions

# # Menggabungkan dataset transactions dan product berdasarkan kolom 'product_id' dan 'id'
# merged_data = transactions.merge(product, left_on='product_id', right_on='product_id', how='inner')

# # Menghapus kolom yang tidak diperlukan dari DataFrame merged_data
# merged_data.drop(['product_id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'season', 'year_y', 'usage'], axis=1, inplace=True)

# # Menampilkan hasil gabungan
# merged_data[['customer_id','payment_method','payment_status','promo_code','productDisplayName']]

In [ ]:
# # Groupby session_id dan traffic_source
# grouped_data = click.groupby(['session_id', 'traffic_source']).size().unstack(fill_value=0)

# # Tampilkan hasil groupby
# print("Hasil Groupby dengan Lalu Lintas Sumber:")
# print(grouped_data)

In [ ]:
# click['session_id'].nunique()

In [ ]:
# # Ekstrak Tahun dari kolom 'created_at'
# transactions['month'] = transactions['created_at'].dt.to_period('M')

# # Groupby berdasarkan Tahun dan promo_code, lalu hitung jumlah penggunaan promo_code
# grouped_data = transactions.groupby(['month', 'promo_code']).size().unstack(fill_value=0)

# # Membuat time series plot
# grouped_data.plot(kind='line', marker='o', figsize=(10, 6))
# plt.xlabel('Tahun')
# plt.ylabel('Jumlah Penggunaan Promo Code')
# plt.title('Penggunaan Promo Code per Tahun')
# plt.legend(title='Promo Code', title_fontsize='12')
# plt.show()

# **3. Feature Engineering**

## **Feature Selection**

In [ ]:
click.event_name.unique()

In [ ]:
#Mengelompokan Berdasarkan Kegiatan Yang Ada di Event Name Dan Membuat Feature Baru

# Homepage
homepage = click[(click['event_name'] == 'HOMEPAGE')]
homepage = homepage.groupby('session_id')['event_name'].count()
homepage = homepage.to_frame().rename(columns={"event_name": "homepage"}).reset_index()

# Scroll
scroll = click[(click['event_name'] == 'SCROLL')]
scroll = scroll.groupby('session_id')['event_name'].count()
scroll = scroll.to_frame().rename(columns={"event_name": "scroll"}).reset_index()

#Search
search = click[(click['event_name'] == 'SEARCH')]
search = search.groupby('session_id')['event_name'].count()
search = search.to_frame().rename(columns={"event_name": "search"}).reset_index()

# Item Detail
item_detail = click[(click['event_name'] == 'ITEM_DETAIL')]
item_detail = item_detail.groupby('session_id')['event_name'].count()
item_detail = item_detail.to_frame().rename(columns={"event_name": "item_detail"}).reset_index()

# Add To Cart
add_to_cart = click[(click['event_name'] == 'ADD_TO_CART')]
add_to_cart = add_to_cart.groupby('session_id')['event_name'].count()
add_to_cart = add_to_cart.to_frame().rename(columns={"event_name": "add_to_cart"}).reset_index()

# Add Promo
add_promo = click[(click['event_name'] == 'ADD_PROMO')]
add_promo = add_promo.groupby('session_id')['event_name'].count()
add_promo = add_promo.to_frame().rename(columns={"event_name": "add_promo"}).reset_index()

# click
click_event = click[(click['event_name'] == 'CLICK')]
click_event = click_event.groupby('session_id')['event_name'].count()
click_event = click_event.to_frame().rename(columns={"event_name": "click"}).reset_index()

# promo_page
promo_page = click[(click['event_name'] == 'PROMO_PAGE')]
promo_page = promo_page.groupby('session_id')['event_name'].count()
promo_page = promo_page.to_frame().rename(columns={"event_name": "promo_page"}).reset_index()

# Booking
booking = click[(click['event_name'] == 'BOOKING')]
booking = booking.groupby('session_id')['event_name'].count()
booking = booking.to_frame().rename(columns={"event_name": "booking"}).reset_index()

In [ ]:
# Menggabungkan Semua Feature Diatas

click_session = click[['session_id', 'event_time']].merge(homepage, on='session_id', how='left')
click_session = click_session.merge(scroll, on='session_id', how='left')
click_session = click_session.merge(search, on='session_id', how='left')
click_session = click_session.merge(item_detail, on='session_id', how='left')
click_session = click_session.merge(add_to_cart, on='session_id', how='left')
click_session = click_session.merge(add_promo, on='session_id', how='left')
click_session = click_session.merge(click_event, on='session_id', how='left')
click_session = click_session.merge(promo_page, on='session_id', how='left')
click_session = click_session.merge(booking, on='session_id', how='left')

- HOMEPAGE: Pengguna masuk ke halaman utama platform saat mereka mulai sesi.
- SCROLL: Pengguna menggulir halaman untuk menjelajahi berbagai produk atau konten.
- SEARCH: Pengguna melakukan pencarian untuk produk tertentu atau kategori.
- ITEM_DETAIL: Pengguna mengklik pada item atau produk tertentu untuk melihat detailnya.
- ADD_TO_CART: Pengguna menambahkan item ke keranjang belanja.
- ADD_PROMO: Pengguna mungkin menambahkan promo atau kode diskon ke pesanan mereka.
- CLICK: Pengguna melakukan klik pada berbagai elemen, mungkin termasuk tautan atau gambar.
- PROMO_PAGE: Pengguna mengunjungi halaman promosi atau diskon khusus.

In [ ]:
click_session.head()

Disini terlihat bahwa dalam satu session customer melakukan banyak aktivitas pada waktu yang berbeda

## Master Dataset

In [ ]:
# trans = transactions.copy()
# master_dataset = trans.merge(click_session, on='session_id', how='left')
# master_dataset.head()

In [ ]:
# # Download Master Dataset
# master_dataset.to_csv('master.csv', index=False)
# files.download("master.csv")

## Feature Dataset

Menggabungkan dataset transaction (sukses saja) dengan click_session untuk melihat aktivitas dan informasi dari customer saat melakukan pemesanan

In [ ]:
# Menggabungkan Transaction (Sukses Saja) dengan Feature Selection Sebelumnya
trans_success = transactions[transactions['payment_status'] == 'Success'][['session_id', 'created_at', 'customer_id', 'booking_id', 'promo_amount', 'shipment_fee', 'total_amount', 'product_id', 'quantity', 'item_price']]

feature = trans_success.merge(click_session, on='session_id', how='left')
feature.head()

In [ ]:
# Melakukan agregasi untuk memahami perilaku pelanggan secara keseluruhan.

agg = feature.groupby(['customer_id']).agg(

#Melihat Frekuensi dan Total Uniq Produk yang pernah dibeli oleh Customer
first_transaction = pd.NamedAgg(column = 'created_at', aggfunc=min),
last_transaction = pd.NamedAgg(column = 'created_at', aggfunc=max),
freq_transaction = pd.NamedAgg(column = 'booking_id', aggfunc='nunique'),
total_unique_product = pd.NamedAgg(column = 'product_id', aggfunc='nunique'),

#Melihat Jumlah Produk yang pernah dibeli Customer
total_quantity = pd.NamedAgg(column = 'quantity', aggfunc=sum),
avg_quantity = pd.NamedAgg(column = 'quantity', aggfunc='mean'),

#Melihat Jumlah pengeluaran yang pernah dilakukan customer
total_amount = pd.NamedAgg(column = 'total_amount', aggfunc=sum),
avg_amount = pd.NamedAgg(column = 'total_amount', aggfunc='mean'),

#Melihat Item Price yang pernah dibeli Customer
min_item_price = pd.NamedAgg(column = 'item_price', aggfunc=min),
max_item_price = pd.NamedAgg(column = 'item_price', aggfunc=max),
avg_item_price = pd.NamedAgg(column = 'item_price', aggfunc='mean'),

#Melihat Jumlah potongan Promo yang sudah pernah dipakai Customer
total_promo_amount = pd.NamedAgg(column = 'promo_amount', aggfunc=sum),
total_promo_code = pd.NamedAgg(column = 'promo_amount', aggfunc='nunique'),

#Melihat Total Biaya pengiriman yang pernah dibayarkan Customer
total_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=sum),
min_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=min),
max_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=max),
avg_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc='mean'),

#Melihat Rata-Rata Aktivitas yang pernah dilakukan Customer
avg_homepage = pd.NamedAgg(column = 'homepage', aggfunc='mean'),
avg_scroll = pd.NamedAgg(column = 'scroll', aggfunc='mean'),
avg_click = pd.NamedAgg(column = 'click', aggfunc='mean'),
item_detail = pd.NamedAgg(column = 'item_detail', aggfunc='mean'),
avg_promo_page = pd.NamedAgg(column = 'promo_page', aggfunc='mean'),
avg_search = pd.NamedAgg(column = 'search', aggfunc='mean'),
avg_add_to_cart = pd.NamedAgg(column = 'add_to_cart', aggfunc='mean'),
avg_add_promo = pd.NamedAgg(column = 'add_promo', aggfunc='mean'),

).reset_index()


In [ ]:
agg.head()

## **Churn Label**

In [ ]:
# Melihat rentang waktu dalam kolom 'created_at'.
print(min(feature['created_at']))
print(max(feature['created_at']))

In [ ]:
# Membuat Fungsi untuk label Churn (Perbulan)

# Per-1 Bulan
def label(df):
    if (df['yearmonth_x'] == 0) or (df['yearmonth_y'] == 0):
        return 1  # 1 adalah customer label churn
    elif (df['yearmonth_x'] != 0) and (df['yearmonth_y'] != 0):
        return 0  # 0 adalah customer level tidak churn
    else:
        return " "

# Per-6 Bulan
def sixlabel(df):
    if (df['6Month_x'] == 0) or (df['6Month_y'] == 0):      #6Month_x : observasi window
        return 1  # 1 adalah customer label churn           #6Month_y : perform window
    elif (df['6Month_x'] != 0) and (df['6Month_y'] != 0):
        return 0  # 0 adalah customer level tidak churn
    else:
        return " "

### One Month

In [ ]:
 #Mengubah format tanggal dan waktu pada create_at menjadi string
feature['yearmonth'] = feature['created_at'].dt.strftime('%Y-%m')

In [ ]:
print(min(feature['yearmonth']))
print(max(feature['yearmonth']))

In [ ]:
# # Melakukan Agregasi Untuk memahami perilaku pelanggan secara keseluruhan.
# agg = feature.groupby(['yearmonth','customer_id']).agg(

# # Melihat Frekuensi dan Total Uniq Produk yang pernah dibeli oleh Customer
# first_transaction = pd.NamedAgg(column = 'created_at', aggfunc=min),
# last_transaction = pd.NamedAgg(column = 'created_at', aggfunc=max),
# freq_transaction = pd.NamedAgg(column = 'booking_id', aggfunc='nunique'),
# total_unique_product = pd.NamedAgg(column = 'product_id', aggfunc='nunique'),

# # Melihat Jumlah Produk yang pernah dibeli Customer
# total_quantity = pd.NamedAgg(column = 'quantity', aggfunc=sum),
# avg_quantity = pd.NamedAgg(column = 'quantity', aggfunc='mean'),

# # Melihat Jumlah pengeluaran yang pernah dilakukan customer
# total_amount = pd.NamedAgg(column = 'total_amount', aggfunc=sum),
# avg_amount = pd.NamedAgg(column = 'total_amount', aggfunc='mean'),

# # Melihat Item Price yang pernah dibeli Customer
# min_item_price = pd.NamedAgg(column = 'item_price', aggfunc=min),
# max_item_price = pd.NamedAgg(column = 'item_price', aggfunc=max),
# avg_item_price = pd.NamedAgg(column = 'item_price', aggfunc='mean'),

# # Melihat Jumlah potongan Promo yang sudah pernah dipakai Customer
# total_promo_amount = pd.NamedAgg(column = 'promo_amount', aggfunc=sum),
# total_promo_code = pd.NamedAgg(column = 'promo_amount', aggfunc='nunique'),

# # Melihat Total Biaya pengiriman yang pernah dibayarkan Customer
# total_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=sum),
# min_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=min),
# max_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=max),
# avg_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc='mean'),

# # Melihat Rata-Rata Aktivitas yang pernah dilakukan Customer
# avg_homepage = pd.NamedAgg(column = 'homepage', aggfunc='mean'),
# avg_scroll = pd.NamedAgg(column = 'scroll', aggfunc='mean'),
# avg_click = pd.NamedAgg(column = 'click', aggfunc='mean'),
# item_detail = pd.NamedAgg(column = 'item_detail', aggfunc='mean'),
# avg_promo_page = pd.NamedAgg(column = 'promo_page', aggfunc='mean'),
# avg_search = pd.NamedAgg(column = 'search', aggfunc='mean'),
# avg_add_to_cart = pd.NamedAgg(column = 'add_to_cart', aggfunc='mean'),
# avg_add_promo = pd.NamedAgg(column = 'add_promo', aggfunc='mean'),

# ).reset_index()

In [ ]:
# agg.head()

### Six Month

In [ ]:
# Membuat Kolom 6Month berisi data (create_at)
feature['6Month'] = feature['created_at'].dt.year.astype(str) + 'S' + np.where(feature['created_at'].dt.quarter.gt(2), 2, 1).astype(str)

Membuat Kolom 6Month yang menggambarkan data ini masuk ke semester 1 (6 bulan pertama) atau semester 2 (6 bulan terakhir), berdasarkan informasi kuartal dari kolom 'created_at'. Jika < kuartal 2 berarti masuk kedalam semester 2 (6 bulan terakhir)

In [ ]:
# Melakukan Agregasi Untuk memahami perilaku pelanggan secara keseluruhan.

agg = feature.groupby(['6Month','customer_id']).agg(

#Melihat Frekuensi dan Total Uniq Produk yang pernah dibeli oleh Customer
first_transaction = pd.NamedAgg(column = 'created_at', aggfunc=min),
last_transaction = pd.NamedAgg(column = 'created_at', aggfunc=max),
freq_transaction = pd.NamedAgg(column = 'booking_id', aggfunc='nunique'),
total_unique_product = pd.NamedAgg(column = 'product_id', aggfunc='nunique'),

#Melihat Jumlah Produk yang pernah dibeli Customer
total_quantity = pd.NamedAgg(column = 'quantity', aggfunc=sum),
avg_quantity = pd.NamedAgg(column = 'quantity', aggfunc='mean'),

#Melihat Jumlah pengeluaran yang pernah dilakukan customer
total_amount = pd.NamedAgg(column = 'total_amount', aggfunc=sum),
avg_amount = pd.NamedAgg(column = 'total_amount', aggfunc='mean'),

#Melihat Item Price yang pernah dibeli Customer
min_item_price = pd.NamedAgg(column = 'item_price', aggfunc=min),
max_item_price = pd.NamedAgg(column = 'item_price', aggfunc=max),
avg_item_price = pd.NamedAgg(column = 'item_price', aggfunc='mean'),

#Melihat Jumlah potongan Promo yang sudah pernah dipakai Customer
total_promo_amount = pd.NamedAgg(column = 'promo_amount', aggfunc=sum),
total_promo_code = pd.NamedAgg(column = 'promo_amount', aggfunc='nunique'),

#Melihat Total Biaya pengiriman yang pernah dibayarkan Customer
total_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=sum),
min_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=min),
max_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc=max),
avg_shipment = pd.NamedAgg(column = 'shipment_fee', aggfunc='mean'),

#Melihat Rata-Rata Aktivitas yang pernah dilakukan Customer
avg_homepage = pd.NamedAgg(column = 'homepage', aggfunc='mean'),
avg_scroll = pd.NamedAgg(column = 'scroll', aggfunc='mean'),
avg_click = pd.NamedAgg(column = 'click', aggfunc='mean'),
item_detail = pd.NamedAgg(column = 'item_detail', aggfunc='mean'),
avg_promo_page = pd.NamedAgg(column = 'promo_page', aggfunc='mean'),
avg_search = pd.NamedAgg(column = 'search', aggfunc='mean'),
avg_add_to_cart = pd.NamedAgg(column = 'add_to_cart', aggfunc='mean'),
avg_add_promo = pd.NamedAgg(column = 'add_promo', aggfunc='mean'),

).reset_index()


In [ ]:
agg.head()

####Outlier

In [ ]:
agg.info()

In [ ]:
# # mengecek outlier
# numeric_columns = agg.select_dtypes(include=['int', 'float']).columns
# fig, axs = plt.subplots(ncols=len(numeric_columns), figsize=(50, 15))

# for i, column in enumerate(numeric_columns):
#     sns.boxplot(y=column, data=agg, ax=axs[i], whis=1.5)

#     axs[i].set_title(f'Box Plot for {column}')
#     axs[i].set_xlabel('')

# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
# plt.show()



In [ ]:
# # Calculate Q1 and Q3 for the selected numeric columns
# Q1 = agg[numeric_columns].quantile(0.25)
# Q3 = agg[numeric_columns].quantile(0.75)

# # Calculate IQR, maximum, and minimum for outlier handling
# IQR = Q3 - Q1
# maximum = Q3 + (1.5 * IQR)
# minimum = Q1 - (1.5 * IQR)

# # Handle outliers by capping values that exceed the maximum or fall below the minimum
# agg[numeric_columns] = agg[numeric_columns].clip(lower=minimum, upper=maximum, axis=1)

# # Display the agg distribution after outlier handling
# print('\nagg Distribution after Outliers being Handled:')
# print(agg[numeric_columns].describe())

In [ ]:
# print(agg[numeric_columns].describe())

In [ ]:
# # mengecek outlier
# numeric_columns = agg.select_dtypes(include=['int', 'float']).columns
# fig, axs = plt.subplots(ncols=len(numeric_columns), figsize=(50, 15))

# for i, column in enumerate(numeric_columns):
#     sns.boxplot(y=column, data=agg, ax=axs[i], whis=1.5)

#     axs[i].set_title(f'Box Plot for {column}')
#     axs[i].set_xlabel('')

# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
# plt.show()

In [ ]:
# # Membagi DF agg sesuai dengan semester(6 bulan) tertentu dalam rentang waktu dari tahun 2015 hingga 2022, berdasarkan kolom '6Month'

sixmnt2_15 = agg[agg['6Month'] == '2015S2']

# 2016

sixmnt1_16 = agg[agg['6Month'] == '2016S1'] #6bulan pertama di 2016
sixmnt2_16 = agg[agg['6Month'] == '2016S2'] #6bulan kedua di 2016

# 2017

sixmnt1_17 = agg[agg['6Month'] == '2017S1']
sixmnt2_17 = agg[agg['6Month'] == '2017S2']

# 2018

sixmnt1_18 = agg[agg['6Month'] == '2018S1']
sixmnt2_18 = agg[agg['6Month'] == '2018S2']

# 2019

sixmnt1_19 = agg[agg['6Month'] == '2019S1']
sixmnt2_19 = agg[agg['6Month'] == '2019S2']

# 2020

sixmnt1_20 = agg[agg['6Month'] == '2020S1']
sixmnt2_20 = agg[agg['6Month'] == '2020S2']

# 2021

sixmnt1_21 = agg[agg['6Month'] == '2021S1']
sixmnt2_21 = agg[agg['6Month'] == '2021S2']

# 2022

sixmnt1_22 = agg[agg['6Month'] == '2022S1']
sixmnt2_22 = agg[agg['6Month'] == '2022S2']

In [ ]:
sixmnt1_22

In [ ]:
# 2016

# 2016
churn1_16 = sixmnt2_15.merge(sixmnt1_16[['customer_id', '6Month']], on='customer_id', how='outer')
churn1_16.fillna(0, inplace = True)
churn1_16['churn'] = churn1_16.apply(sixlabel, axis = 1)

# semester2_2016
churn2_16 = sixmnt1_16.merge(sixmnt2_16[['customer_id', '6Month']], on='customer_id', how='outer')
churn2_16.fillna(0, inplace = True)
churn2_16['churn'] = churn1_16.apply(sixlabel, axis = 1)

# 2017

# semester1_2017
churn1_17 = sixmnt2_16.merge(sixmnt1_17[['customer_id', '6Month']], on='customer_id', how='outer')
churn1_17.fillna(0, inplace = True)
churn1_17['churn'] = churn1_17.apply(sixlabel, axis = 1)

# semester2_2017
churn2_17 = sixmnt1_17.merge(sixmnt2_17[['customer_id', '6Month']], on='customer_id', how='outer')
churn2_17.fillna(0, inplace = True)
churn2_17['churn'] = churn2_17.apply(sixlabel, axis = 1)

# 2018

# semester1_2018
churn1_18 = sixmnt2_17.merge(sixmnt1_18[['customer_id', '6Month']], on='customer_id', how='outer')
churn1_18.fillna(0, inplace = True)
churn1_18['churn'] = churn1_18.apply(sixlabel, axis = 1)

# semester2_2018
churn2_18 = sixmnt1_18.merge(sixmnt2_18[['customer_id', '6Month']], on='customer_id', how='outer')
churn2_18.fillna(0, inplace = True)
churn2_18['churn'] = churn2_18.apply(sixlabel, axis = 1)

# 2019

# semester1_2019
churn1_19 = sixmnt2_18.merge(sixmnt1_19[['customer_id', '6Month']], on='customer_id', how='outer')
churn1_19.fillna(0, inplace = True)
churn1_19['churn'] = churn1_19.apply(sixlabel, axis = 1)

# semester2_2019
churn2_19 = sixmnt1_19.merge(sixmnt2_19[['customer_id', '6Month']], on='customer_id', how='outer')
churn2_19.fillna(0, inplace = True)
churn2_19['churn'] = churn2_19.apply(sixlabel, axis = 1)

# 2020

# semester1_2020
churn1_20 = sixmnt2_19.merge(sixmnt1_20[['customer_id', '6Month']], on='customer_id', how='outer')
churn1_20.fillna(0, inplace = True)
churn1_20['churn'] = churn1_20.apply(sixlabel, axis = 1)

# semester2_2020
churn2_20 = sixmnt1_20.merge(sixmnt2_20[['customer_id', '6Month']], on='customer_id', how='outer')
churn2_20.fillna(0, inplace = True)
churn2_20['churn'] = churn2_20.apply(sixlabel, axis = 1)

# 2021

# semester1_2021
churn1_21 = sixmnt2_20.merge(sixmnt1_21[['customer_id', '6Month']], on='customer_id', how='outer')
churn1_21.fillna(0, inplace = True)
churn1_21['churn'] = churn1_21.apply(sixlabel, axis = 1)

# semester2_2021
churn2_21 = sixmnt1_21.merge(sixmnt2_21[['customer_id', '6Month']], on='customer_id', how='outer')
churn2_21.fillna(0, inplace = True)
churn2_21['churn'] = churn2_21.apply(sixlabel, axis = 1)

# 2022

# semester1_2022
churn1_22 = sixmnt2_21.merge(sixmnt1_22[['customer_id', '6Month']], on='customer_id', how='outer')
churn1_22.fillna(0, inplace = True)
churn1_22['churn'] = churn1_22.apply(sixlabel, axis = 1)

# semester2_2022
churn2_22 = sixmnt1_22.merge(sixmnt2_22[['customer_id', '6Month']], on='customer_id', how='outer')
churn2_22.fillna(0, inplace = True)
churn2_22['churn'] = churn2_22.apply(sixlabel, axis = 1)

In [ ]:
# Menggabungkan Label Churn yang telah dibuat
# data2016_6month = churn1_16
data2017_6month = pd.concat([churn1_17, churn2_17]).reset_index(drop=True)
data2018_6month = pd.concat([churn1_18, churn2_18]).reset_index(drop=True)
data2019_6month = pd.concat([churn1_19, churn2_19]).reset_index(drop=True)
data2020_6month = pd.concat([churn1_20, churn2_20]).reset_index(drop=True)
data2021_6month = pd.concat([churn1_21, churn2_21]).reset_index(drop=True)
data2022_6month = pd.concat([churn1_22, churn2_22]).reset_index(drop=True)

In [ ]:
# Menggabungkan Semua Data
final_6month = pd.concat([data2017_6month,data2018_6month,data2019_6month,data2020_6month,data2021_6month,data2022_6month])
final_6month.head()

In [ ]:
final = final_6month.copy()

In [ ]:
final.info()

In [ ]:

# Pastikan kolom 'churn' bukan indeks dan memiliki tipe data yang benar
final.reset_index(inplace=True)
final['churn'] = final['churn'].astype(str)  # Konversi ke string jika belum

# Buat countplot
plt.figure(figsize=(6, 5))
sns.countplot(data=final, x='churn')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Hitung jumlah churn dan no churn
churn_counts = final['churn'].value_counts()

# Label dan ukuran untuk pie chart
labels = 'Churn', 'No Churn'
sizes = [churn_counts[1], churn_counts[0]]
explode = (0, 0.1)

# Buat pie chart menggunakan Matplotlib dengan gaya Seaborn
plt.figure(figsize=(6, 4))
sns.set(style="whitegrid")
sns.color_palette("pastel")
plt.pie(sizes, labels=labels, explode=explode, autopct='%1.1f%%', shadow=True, startangle=90)
plt.title("Proportion of Customer Churn and No Churn", size=20)
plt.axis('equal')  # Memastikan pie chart berbentuk lingkaran
plt.show()


In [ ]:
# counts = master.groupby(['customer_id']).created_at.nunique()
# counts.name = 'Number of Previous Visits'

In [ ]:
# y_var = master.groupby(['customer_id']).created_at.max() < '2022-04-30'
# y_var.name = 'Churned'

In [ ]:
# Dataset = counts.to_frame().join(y_var)
# Dataset

In [ ]:
# # Membagi dataset menjadi data training dan data testing
# X = Dataset['Number of Previous Visits'].values.reshape(-1,1)
# y = Dataset['Churned']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Membuat model decision tree
# clf = DecisionTreeClassifier()

# # Melatih model dengan data training
# clf.fit(X_train, y_train)

# # Memprediksi data testing
# y_pred = clf.predict(X_test)

# # Menghitung akurasi model
# accuracy = accuracy_score(y_test, y_pred)

# print(f"Akurasi model: {accuracy}")

# **4. Machine Learning**

###Baseline Model

In [ ]:
X = final_6month[['freq_transaction','freq_transaction','total_unique_product','total_quantity']]
y = final_6month['churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()

X_train= X_train.values.reshape(-1, 1)
X_test= X_test.values.reshape(-1, 1)
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

###Hyperparameter Tunning

###Cross Validation